**Note:** running with on purpose created environment `r5py-env`<br/>
(conflicts with geopandas, still WIP)

script from here https://r5py.readthedocs.io/en/stable/notebooks/basic-usage.html

## Data import and variables/path definition

In [1]:
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime
import tracc
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode
sys.argv.append(["--max-memory", "8G"])

points_file = "./../data/gis/tynewear_lsoas_centroids_wgs84.shp"
jobs_file = "./../data/bres_jobs_tynewear.csv"
gs_accesspoints_file = "./../data/gis/tynewear_gs_accesspoints_lsoas.csv"

In [2]:
# reading in the origin/destination points shp (just for tynewear region):
centroids = gpd.read_file(points_file)
centroids['id'] = centroids['LSOA21CD']
centroids.head()

# reading in land use data:
jobs_per_lsoa = pd.read_csv(jobs_file)
jobs_per_lsoa.head()

accesspoints_per_lsoa = pd.read_csv(gs_accesspoints_file)
accesspoints_per_lsoa.head()

,LSOA11CD,n_access
0,E01000001,3
1,E01000002,1
2,E01000003,2
3,E01000005,9
4,E01000006,0


In [4]:
centroids.explore()

In [5]:
accesspoints_per_lsoa.n_access.dtype

dtype('int64')

In [6]:
# merging geospatial data (admin boundaries/centroids) with landuse data (jobs and greenspaces, could be popuplation etc)

landuse_gdf = pd.merge(centroids, # gdf file
                       jobs_per_lsoa, # info we want to add
                       how = "inner", # this allows to pick (from the nationals centroids table) only LSOAs that belong to the region (jobs file)
                       left_on = "LSOA21CD",
                       right_on = "LSOA11CD")
landuse_gdf.head()

landuse_gdf = pd.merge(landuse_gdf,
                       accesspoints_per_lsoa,
                       how = "inner",
                       left_on = "LSOA21CD",
                       right_on = "LSOA11CD")
landuse_gdf.head()

landuse_gdf.jobs = landuse_gdf.jobs.fillna(0).astype(np.int64) # removing NaN and changing type to Int
# hoping that this makes "explore" work below. NOTE: deal with this issue later on in a better way (IE clean data before importing them in python)
landuse_gdf.n_access = landuse_gdf.n_access.fillna(0).astype(np.int64)
landuse_gdf.dtypes



LSOA21CD        object
geometry      geometry
id              object
LSOA11CD_x      object
jobs             int64
LSOA11CD_y      object
n_access         int64
dtype: object

In [7]:
landuse_gdf.n_access.isnull().sum().sum() # (checking number of na in column)

0

In [8]:
landuse_gdf.explore(column="n_access",
                    cmap="plasma"
                    # style_kwds={"style_function":lambda x: {"radius":x["properties"]["jobs"]}} # doesnt work for me
                    )

## Load transport network

>  To import the street and public transport networks, instantiate an `r5py.TransportNetwork` with the file paths to the OSM extract and the GTFS files:

In [9]:
transport_network = TransportNetwork(
    "/Users/azanchetta/OTP/graphs/tynewear15/tyne-and-wear-latest.osm.pbf",
    [
        "/Users/azanchetta/OTP/graphs/tynewear15/itm_north_east_gtfs.zip"
    ]
)

## Compute travel matrix
1. generate time travel matrix from/to all centroids

In [11]:
travel_time_matrix_computer_transit = TravelTimeMatrixComputer(
    transport_network,
    origins=centroids,
    departure=datetime.datetime(2023,1,19,8,30),
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK]
)
travel_time_matrix_centroids = travel_time_matrix_computer_transit.compute_travel_times()
travel_time_matrix_centroids.head()

NameError: name 'travel_time_matrix_computer' is not defined

2. generate _detailed_ travel matrix
> When `breakdown` is enabled, r5py produces information about the used routes for each origin-destination pair, as well as total time disaggregated by access, waiting, in-vehicle and transfer times



Note: can specify option for detailed matrix as in https://r5py.readthedocs.io/en/stable/reference.html:
>`transport_modes (list[r5py.TransitMode | r5py.LegMode])` – The mode of transport to use for routing. Default: [r5py.TransitMode.TRANSIT] (all public transport)<br/>
`access_modes (list[r5py.LegMode])` – Mode of transport to public transport stops. Default: [r5py.LegMode.WALK]<br/>
`egress_modes (list[r5py.LegMode])` – Mode of transport from public transport stops. Default: access_modes


In [10]:
# travel_time_matrix_computer = TravelTimeMatrixComputer(
#     transport_network,
#     origins=centroids,
#     departure=datetime.datetime(2023,1,19,8,30),
#     transport_modes=[TransitMode.TRANSIT, LegMode.WALK],
#     breakdown=True,
# )
# travel_time_matrix_detailed = travel_time_matrix_computer.compute_travel_times()
# travel_time_matrix_detailed.head()

# run for specific needed analysis - not ready yet
# when run on all the LSOAs for Tyne and Wear it takes

In [ ]:
# exporting output
travel_time_matrix_detailed.to_csv("./../output/detailed_ttm_tynewear_transit.csv")

Compute accessibility

Using here `tracc` or `pysal.access` (link)[https://pysal.org/access/index.html]